## Cadences Network Visualization with Louvain Communities



In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


# B.  Select Corpus

* The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
* The pieces are provided as a **list**, within square brackets and separated by commas.  
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: `corpus CorpusBase(
       ['https://crimproject.org/mei/CRIM_Mass_0006_1.mei',
       'https://crimproject.org/mei/CRIM_Mass_0006_2.mei',
       'https://crimproject.org/mei/CRIM_Mass_0006_3.mei'])`
       
* Read the documentation:  `print(CorpusBase.batch.__doc__)`


In [29]:
corpus = CorpusBase(['/Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Model_0019.mei',
                     '/Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_1.mei',
                     '/Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_2.mei',
                     '/Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_3.mei'
                    ,'/Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_4.mei',
                    '/Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_5.mei'
                    ])
                
#                     'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
#                      'https://crimproject.org/mei/CRIM_Mass_0019_3.mei'
#                     ,'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
#                     'https://crimproject.org/mei/CRIM_Mass_0019_5.mei'])

Previously imported piece detected.
Successfully imported /Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_1.mei
Successfully imported /Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_2.mei
Successfully imported /Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_3.mei
Successfully imported /Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_4.mei
Successfully imported /Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Mass_0019_5.mei


In [2]:
# Just one piece

corpus = CorpusBase(['/Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Model_0019.mei'])

Successfully imported /Users/rfreedma/Documents/CRIM_Python/CRIM-online_new/crim/static/mei/MEI_4.0/CRIM_Model_0019.mei


In [30]:


list_of_dfs = corpus.batch(ImportedPiece.cadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
combined_df.dropna(inplace=True)
combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone']  + "_" + combined_df['Beat'].apply(str)
combined_df['some_features'] = combined_df['CadType'] + "_" + combined_df['Tone']
combined_df.copy
combined_df["previous_all_feat"] = combined_df.groupby("Title")['all_features'].shift(1)
combined_df["previous_some_feat"] = combined_df.groupby("Title")['some_features'].shift(1)
combined_df

,CadType,LeadingTones,CVFs,Low,RelLow,Tone,RelTone,TSig,Measure,Beat,Sounding,Progress,SinceLast,ToNext,Composer,Title,all_features,some_features,previous_all_feat,previous_some_feat
0,Clausula Vera,1.0,TCu,G3,P1,G,P8,4/2,7,1.0,4.0,0.090909,48.0,36.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Clausula Vera_G_1.0,Clausula Vera_G,NaN,NaN
1,Evaded Clausula Vera,1.0,tC,G3,P1,G,P8,4/2,11,3.0,3.0,0.159091,36.0,68.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Evaded Clausula Vera_G_3.0,Evaded Clausula Vera_G,Clausula Vera_G_1.0,Clausula Vera_G
2,Authentic,1.0,TCB,G3,P1,G,P8,4/2,20,1.0,4.0,0.287879,68.0,72.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Authentic_G_1.0,Authentic_G,Evaded Clausula Vera_G_3.0,Evaded Clausula Vera_G
3,Evaded Clausula Vera,1.0,zC,F3,-M2,C,P4,4/2,29,1.0,3.0,0.424242,72.0,28.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Evaded Clausula Vera_C_1.0,Evaded Clausula Vera_C,Authentic_G_1.0,Authentic_G
4,Evaded Clausula Vera,1.0,tC,C4,P4,C,P4,4/2,32,3.0,3.0,0.477273,28.0,34.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Evaded Clausula Vera_C_3.0,Evaded Clausula Vera_C,Evaded Clausula Vera_C_1.0,Evaded Clausula Vera_C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Authentic,0.0,CB,A3,M2,A,M2,4/2,57,3.0,4.0,0.809859,4.0,12.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Authentic_A_3.0,Authentic_A,Phrygian Clausula Vera_E_1.0,Phrygian Clausula Vera_E
156,Clausula Vera,0.0,CT,D3,-P4,D,P5,4/2,62,3.0,4.0,0.880282,4.0,24.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Clausula Vera_D_3.0,Clausula Vera_D,Authentic_A_3.0,Authentic_A
157,Clausula Vera,0.0,CT,D3,-P4,D,P5,4/2,65,3.0,4.0,0.922535,24.0,8.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Clausula Vera_D_3.0,Clausula Vera_D,Clausula Vera_D_3.0,Clausula Vera_D
158,Authentic,0.0,CB,D3,-P4,D,P5,4/2,66,3.0,5.0,0.936620,8.0,12.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Authentic_D_3.0,Authentic_D,Clausula Vera_D_3.0,Clausula Vera_D


### Network Visualization with Louvain Communities

* We first create a list of **successive cadences** in each piece.  These are **Cadence Pairs**.
* Each item in each pair records the following features:  **type**, **tone**, **evaded status**, and **beat strength of last note in the cadence**
* The pairs are thus highly distinct--consisting of four dimensions of two cadences each, such as: `('Clausula Vera_G_False_1.0', 'Clausula Vera_G_True_3.0')`
* This will create a varied network.

In [33]:
# combined_df2 = combined_df.dropna(subset=['previous_all_feat', 'all_features'])
combined_df2 = combined_df.dropna(subset=['some_features'])
combined_df2 = combined_df2.fillna('-')
cadence_pairs = list(zip(combined_df2.previous_some_feat, combined_df2.some_features))
len(cadence_pairs)

118

In [34]:
combined_df2

,CadType,LeadingTones,CVFs,Low,RelLow,Tone,RelTone,TSig,Measure,Beat,Sounding,Progress,SinceLast,ToNext,Composer,Title,all_features,some_features,previous_all_feat,previous_some_feat
0,Clausula Vera,1.0,TCu,G3,P1,G,P8,4/2,7,1.0,4.0,0.090909,48.0,36.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Clausula Vera_G_1.0,Clausula Vera_G,-,-
1,Evaded Clausula Vera,1.0,tC,G3,P1,G,P8,4/2,11,3.0,3.0,0.159091,36.0,68.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Evaded Clausula Vera_G_3.0,Evaded Clausula Vera_G,Clausula Vera_G_1.0,Clausula Vera_G
2,Authentic,1.0,TCB,G3,P1,G,P8,4/2,20,1.0,4.0,0.287879,68.0,72.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Authentic_G_1.0,Authentic_G,Evaded Clausula Vera_G_3.0,Evaded Clausula Vera_G
3,Evaded Clausula Vera,1.0,zC,F3,-M2,C,P4,4/2,29,1.0,3.0,0.424242,72.0,28.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Evaded Clausula Vera_C_1.0,Evaded Clausula Vera_C,Authentic_G_1.0,Authentic_G
4,Evaded Clausula Vera,1.0,tC,C4,P4,C,P4,4/2,32,3.0,3.0,0.477273,28.0,34.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Evaded Clausula Vera_C_3.0,Evaded Clausula Vera_C,Evaded Clausula Vera_C_1.0,Evaded Clausula Vera_C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Authentic,0.0,CB,A3,M2,A,M2,4/2,57,3.0,4.0,0.809859,4.0,12.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Authentic_A_3.0,Authentic_A,Phrygian Clausula Vera_E_1.0,Phrygian Clausula Vera_E
156,Clausula Vera,0.0,CT,D3,-P4,D,P5,4/2,62,3.0,4.0,0.880282,4.0,24.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Clausula Vera_D_3.0,Clausula Vera_D,Authentic_A_3.0,Authentic_A
157,Clausula Vera,0.0,CT,D3,-P4,D,P5,4/2,65,3.0,4.0,0.922535,24.0,8.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Clausula Vera_D_3.0,Clausula Vera_D,Clausula Vera_D_3.0,Clausula Vera_D
158,Authentic,0.0,CB,D3,-P4,D,P5,4/2,66,3.0,5.0,0.936620,8.0,12.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Authentic_D_3.0,Authentic_D,Clausula Vera_D_3.0,Clausula Vera_D


In [35]:
cadence_pairs

[('-', 'Clausula Vera_G'),
 ('Clausula Vera_G', 'Evaded Clausula Vera_G'),
 ('Evaded Clausula Vera_G', 'Authentic_G'),
 ('Authentic_G', 'Evaded Clausula Vera_C'),
 ('Evaded Clausula Vera_C', 'Evaded Clausula Vera_C'),
 ('Evaded Clausula Vera_C', 'Authentic_C'),
 ('Authentic_C', 'Evaded Clausula Vera_A'),
 ('Evaded Clausula Vera_A', 'Altizans Only_A'),
 ('Altizans Only_A', 'Clausula Vera_D'),
 ('Clausula Vera_D', 'Clausula Vera_A'),
 ('Clausula Vera_A', 'Authentic_D'),
 ('Authentic_D', 'Altizans Only_D'),
 ('Altizans Only_D', 'Authentic_G'),
 ('-', 'Clausula Vera_G'),
 ('Clausula Vera_G', 'Clausula Vera_G'),
 ('Clausula Vera_G', 'Clausula Vera_D'),
 ('Clausula Vera_D', 'Authentic_G'),
 ('Authentic_G', 'Evaded Clausula Vera_G'),
 ('Evaded Clausula Vera_G', 'Evaded Authentic_C'),
 ('Evaded Authentic_C', 'Evaded Authentic_G'),
 ('Evaded Authentic_G', 'Evaded Clausula Vera_C'),
 ('Evaded Clausula Vera_C', 'Authentic_A'),
 ('Authentic_A', 'Clausula Vera_A'),
 ('Clausula Vera_A', 'Clausula Ve

### A Directed Graph of Pairs

* Directed graphs reflect the succession of events, and create a centrum of the most common 'goal' cadence.  Less frequent successions are at the edge of the graph

In [36]:
def create_node_html(node: str, source_df: pd.DataFrame, node_col: str):
    rows = source_df.loc[source_df[node_col] == node].itertuples()
    
    html_lis = []
    
    for r in rows:
        html_lis.append(f"""<li>Composer: {r.Composer}<br>
                                Title: {r.Title}<br>
                                Measure: {r.Measure}</li>""")
        
    html_ul = f"""<ul>{''.join(html_lis)}</ul>"""
        
    return html_ul


def add_nodes_from_edgelist(edge_list: list, 
                               source_df: pd.DataFrame, 
                               graph: nx.Graph,
                               node_col: str):
    
    graph = deepcopy(graph)
    
    node_list = pd.Series(edge_list).apply(pd.Series).stack().unique()
    
    for n in node_list:
        graph.add_node(n, title=create_node_html(n, source_df, node_col))
        
    return graph

In [37]:

pyvis_graph = Network(notebook=False, width="1800", height="1400", bgcolor="black", font_color="white", directed=False)

G = nx.DiGraph()

try:
    G = add_nodes_from_edgelist(edge_list=cadence_pairs, source_df=combined_df2, graph=G, node_col="all_features")
except Exception as e:
    print(e)
    
# G.add_edges_from(unique_pairs)
G.add_edges_from(cadence_pairs)
# G = add_communities(G)
pyvis_graph.from_nx(G)
pyvis_graph.show('Mass_19_cads_simple.html')